<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/L%C3%B6sung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, LpBinary, LpContinuous
from pulp import lpSum, LpStatus, value
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import math

warnings.filterwarnings('ignore')

print("Initialisiere Fleet Optimization Model...")

# ==========================================
# 1. PARAMETER & DATEN (aus CSV & PDF)
# ==========================================

# Zeit-Diskretisierung
NUM_SLOTS = 96
SLOT_DURATION = 0.25  # 15 min

# Kosten & Technische Parameter (PDF & Prompt)
GAS_PRICE = 1.60        # Euro/Liter
ELEC_PRICE = 0.25       # Euro/kWh (Arbeitspreis)
BASE_ELEC_FEE = 1000    # Euro/Jahr (Grundgebühr)
PEAK_FEE = 150          # Euro/kW/Jahr (Leistungspreis)

GRID_BASE = 500         # kW (bestehender Anschluss)
GRID_EXTRA = 500        # kW (Erweiterung)
GRID_EXPAND_COST = 10000 # Euro/Jahr (Erweiterungskosten)

BATT_PWR_COST = 30      # Euro/kW/Jahr (Capex)
BATT_CAP_COST = 350     # Euro/kWh/Jahr (Capex)
BATT_MAINT = 0.02       # Opex = 2% der Investition
BATT_EFF_ROUND = 0.98   # Round-Trip Efficiency
BATT_EFF = math.sqrt(BATT_EFF_ROUND) # One-way Efficiency für Modellierung
MAX_DOD = 0.975         # Max Depth of Discharge
MIN_SOC = 1 - MAX_DOD   # 0.025

DAYS_PER_YEAR = 260
MAX_CHARGERS = 3        # Max Anzahl Stationen
MAX_VEH = 15            # Max Anzahl Fahrzeuge pro Typ (Optimierungsgrenze)

# Zeitfenster
DAY_START = 24          # 06:00 Uhr (24 * 15min)
DAY_END = 72            # 18:00 Uhr (72 * 15min)

# Routen (aus routes.csv)
routes_data = {
    't-4': {'miles': 250, 'toll_miles': 150, 'start': '06:45', 'end': '17:15'},
    't-5': {'miles': 250, 'toll_miles': 150, 'start': '06:30', 'end': '17:00'},
    't-6': {'miles': 250, 'toll_miles': 150, 'start': '06:00', 'end': '16:30'},
    's-1': {'miles': 120, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    's-2': {'miles': 120, 'toll_miles': 32, 'start': '06:00', 'end': '16:00'},
    's-3': {'miles': 120, 'toll_miles': 32, 'start': '09:00', 'end': '16:00'},
    's-4': {'miles': 120, 'toll_miles': 32, 'start': '06:30', 'end': '16:30'},
    'w1':  {'miles': 100, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    'w2':  {'miles': 100, 'toll_miles': 32, 'start': '08:00', 'end': '18:00'},
    'w3':  {'miles': 100, 'toll_miles': 32, 'start': '06:45', 'end': '16:45'},
    'w4':  {'miles': 100, 'toll_miles': 32, 'start': '06:00', 'end': '16:00'},
    'w5':  {'miles': 100, 'toll_miles': 32, 'start': '07:00', 'end': '17:00'},
    'w6':  {'miles': 100, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    'w7':  {'miles': 100, 'toll_miles': 32, 'start': '07:15', 'end': '17:15'},
    'r1':  {'miles': 285, 'toll_miles': 259, 'start': '18:00', 'end': '22:30'},
    'r2':  {'miles': 250, 'toll_miles': 220, 'start': '16:30', 'end': '21:45'},
    'r3':  {'miles': 235, 'toll_miles': 219, 'start': '17:45', 'end': '21:30'},
    'h3':  {'miles': 180, 'toll_miles': 160, 'start': '18:45', 'end': '22:45'},
    'h4':  {'miles': 180, 'toll_miles': 160, 'start': '18:30', 'end': '22:30'},
    'k1':  {'miles': 275, 'toll_miles': 235, 'start': '16:30', 'end': '22:30'},
}

# Fahrzeuge (aus diesel_trucks.csv & electric_trucks.csv)
vehicles_data = {
    'ActrosL': {
        'is_ev': False,
        'lease': 24000,       # Capex
        'maintenance': 6000,  # Opex
        'consumption': 26,    # L/100km
        'tax': 556,           # Kfz-Steuer
        'toll_per_km': 0.34,  # Maut/km
        'battery': None,
        'charge_rate': None,
        'subsidy': 0
    },
    'eActros600': {
        'is_ev': True,
        'lease': 60000,       # Capex
        'maintenance': 6000,  # Opex
        'consumption': 110,   # kWh/100km
        'tax': 0,
        'toll_per_km': 0,
        'battery': 621,       # kWh
        'charge_rate': 400,   # kW
        'subsidy': 1000       # THG-Quote
    },
    'eActros400': {
        'is_ev': True,
        'lease': 50000,       # Capex
        'maintenance': 5000,  # Opex
        'consumption': 105,   # kWh/100km
        'tax': 0,
        'toll_per_km': 0,
        'battery': 414,       # kWh
        'charge_rate': 400,   # kW
        'subsidy': 1000       # THG-Quote
    }
}

# Ladesäulen (aus chargers.csv)
chargers_data = {
    'Alpitronic-50':  {'invest': 3000,  'maint': 1000, 'power': 50,  'cables': 2},
    'Alpitronic-200': {'invest': 10000, 'maint': 1500, 'power': 200, 'cables': 2},
    'Alpitronic-400': {'invest': 16000, 'maint': 2000, 'power': 400, 'cables': 2}
}

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def convert_time_to_slot(t_str):
    """Konvertiert Zeitstring (HH:MM) in 15-Minuten Slot-Index (0-95)"""
    t_str = t_str.strip()
    try:
        # Versuch mit AM/PM
        if 'AM' in t_str or 'PM' in t_str:
            parsed = datetime.strptime(t_str, '%I:%M %p')
        else:
            # Versuch 24h Format
            parsed = datetime.strptime(t_str, '%H:%M')
        mins = parsed.hour * 60 + parsed.minute
        return mins // 15
    except:
        return 0

def check_overlap(r1, r2):
    """Prüft, ob sich zwei Routen zeitlich überschneiden"""
    s1, e1 = routes_data[r1]['t_start'], routes_data[r1]['t_end']
    s2, e2 = routes_data[r2]['t_start'], routes_data[r2]['t_end']
    # Überschneidung, wenn NICHT (Ende1 <= Start2 ODER Ende2 <= Start1)
    return not (e1 <= s2 or e2 <= s1)

def ev_can_do_route(route_id, veh_type):
    """Prüft Reichweiten-Machbarkeit für EV"""
    if not vehicles_data[veh_type]['is_ev']:
        return True
    km = routes_data[route_id]['miles']
    cons = vehicles_data[veh_type]['consumption'] # kWh/100km
    needed = km * cons / 100.0
    cap = vehicles_data[veh_type]['battery']
    # Darf maximal nutzbare Kapazität (DoD) verbrauchen
    return needed <= cap * MAX_DOD

# Datenvorbereitung
for rid in routes_data:
    routes_data[rid]['t_start'] = convert_time_to_slot(routes_data[rid]['start'])
    routes_data[rid]['t_end'] = convert_time_to_slot(routes_data[rid]['end'])
    dur = routes_data[rid]['t_end'] - routes_data[rid]['t_start']
    routes_data[rid]['dur'] = max(dur, 1)

route_ids = list(routes_data.keys())
vehicle_types = list(vehicles_data.keys())
ev_types = [v for v in vehicle_types if vehicles_data[v]['is_ev']]
ice_types = [v for v in vehicle_types if not vehicles_data[v]['is_ev']]
charger_types = list(chargers_data.keys())

# ==========================================
# 3. MODELLIERUNG (Pulp)
# ==========================================

print("Erstelle Optimierungsmodell...")
prob = LpProblem("Fleet_Optimization_CaseStudy", LpMinimize)

# --- Entscheidungsvariablen ---

# Anzahl Fahrzeuge
num_veh = {v: LpVariable(f"num_{v}", lowBound=0, cat=LpInteger) for v in vehicle_types}

# Existenz (binär) für jedes einzelne Fahrzeug k
veh_exists = {}
for v in vehicle_types:
    for k in range(MAX_VEH):
        veh_exists[v,k] = LpVariable(f"exists_{v}_{k}", cat=LpBinary)

# Routenzuordnung (binär)
assign = {}
for r in route_ids:
    for v in vehicle_types:
        for k in range(MAX_VEH):
            assign[r,v,k] = LpVariable(f"assign_{r}_{v}_{k}", cat=LpBinary)

# Ladesäulen Anzahl
num_chg = {c: LpVariable(f"nchg_{c}", lowBound=0, cat=LpInteger) for c in charger_types}

# Netz & Speicher
expand_grid = LpVariable("expand_grid", cat=LpBinary)
batt_pwr = LpVariable("batt_pwr", lowBound=0) # kW
batt_cap = LpVariable("batt_cap", lowBound=0) # kWh
peak_pwr = LpVariable("peak_pwr", lowBound=0) # max Grid Draw

# Speicher Zeitreihen
batt_energy = {t: LpVariable(f"batt_e_{t}", lowBound=0) for t in range(NUM_SLOTS + 1)}
batt_chg = {t: LpVariable(f"batt_ch_{t}", lowBound=0) for t in range(NUM_SLOTS)}
batt_dis = {t: LpVariable(f"batt_dis_{t}", lowBound=0) for t in range(NUM_SLOTS)}

# EV Status & Laden
soc = {}        # State of Charge
ev_chg_pwr = {} # Ladeleistung
at_hub = {}     # Ist am Depot
plugged = {}    # Ist angesteckt

for v in ev_types:
    for k in range(MAX_VEH):
        for t in range(NUM_SLOTS + 1):
             soc[v,k,t] = LpVariable(f"soc_{v}_{k}_{t}", lowBound=MIN_SOC, upBound=1)
        for t in range(NUM_SLOTS):
            ev_chg_pwr[v,k,t] = LpVariable(f"evchg_{v}_{k}_{t}", lowBound=0)
            at_hub[v,k,t] = LpVariable(f"hub_{v}_{k}_{t}", cat=LpBinary)
            plugged[v,k,t] = LpVariable(f"plug_{v}_{k}_{t}", cat=LpBinary)

tot_cables = LpVariable("tot_cables", lowBound=0)
installed_pwr = LpVariable("inst_pwr", lowBound=0)

# --- Zielfunktion (Total Cost of Ownership) ---

# 1. Flottenkosten (Leasing + Wartung + Steuer - Förderung)
fleet_cost = lpSum([
    num_veh[v] * (vehicles_data[v]['lease'] + vehicles_data[v]['maintenance'] + vehicles_data[v]['tax'])
    for v in vehicle_types
])
incentive = lpSum([num_veh[v] * vehicles_data[v]['subsidy'] for v in ev_types])

# 2. Ladeinfrastruktur (Invest + Wartung)
chg_cost = lpSum([
    num_chg[c] * (chargers_data[c]['invest'] + chargers_data[c]['maint'])
    for c in charger_types
])

# 3. Infrastruktur Netz & Speicher
# Netz: Grundgebühr + Optionale Erweiterung
grid_cost = BASE_ELEC_FEE + expand_grid * GRID_EXPAND_COST
# Speicher: Capex (Leistung + Kapazität) + Opex (2% davon) -> Faktor 1.02
batt_cost = (batt_pwr * BATT_PWR_COST + batt_cap * BATT_CAP_COST) * (1 + BATT_MAINT)

# 4. Betriebskosten (Diesel, Strom, Maut, Peak)
# Diesel
fuel_cost = DAYS_PER_YEAR * lpSum([
    assign[r,v,k] * routes_data[r]['miles'] * vehicles_data[v]['consumption'] / 100.0 * GAS_PRICE
    for r in route_ids for v in ice_types for k in range(MAX_VEH)
])
# Strom (Arbeitspreis)
elec_cost = DAYS_PER_YEAR * ELEC_PRICE * lpSum([
    ev_chg_pwr[v,k,t] * SLOT_DURATION
    for v in ev_types for k in range(MAX_VEH) for t in range(NUM_SLOTS)
])
# Peak Load Fee
peak_cost = peak_pwr * PEAK_FEE
# Maut (nur ICE relevant, da EV=0)
toll_cost = DAYS_PER_YEAR * lpSum([
    assign[r,v,k] * routes_data[r]['toll_miles'] * vehicles_data[v]['toll_per_km']
    for r in route_ids for v in ice_types for k in range(MAX_VEH)
])

# Gesamtsumme minimieren
prob += fleet_cost - incentive + chg_cost + grid_cost + batt_cost + fuel_cost + elec_cost + peak_cost + toll_cost

# --- Nebenbedingungen ---

# 1. Flotten & Routen Logik
for r in route_ids:
    # Jede Route genau 1 Fahrzeug
    prob += lpSum([assign[r,v,k] for v in vehicle_types for k in range(MAX_VEH)]) == 1

for v in vehicle_types:
    # Summe Einzelfahrzeuge = Gesamtanzahl
    prob += lpSum([veh_exists[v,k] for k in range(MAX_VEH)]) == num_veh[v]
    # Symmetrie brechen (1 vor 2 füllen)
    for k in range(MAX_VEH - 1):
        prob += veh_exists[v,k] >= veh_exists[v,k+1]

for r in route_ids:
    for v in vehicle_types:
        for k in range(MAX_VEH):
            # Zuordnung nur wenn Fahrzeug existiert
            prob += assign[r,v,k] <= veh_exists[v,k]

# Überschneidungen verhindern
for v in vehicle_types:
    for k in range(MAX_VEH):
        for i, r1 in enumerate(route_ids):
            for r2 in route_ids[i+1:]:
                if check_overlap(r1, r2):
                    prob += assign[r1,v,k] + assign[r2,v,k] <= 1

# EV Reichweite Machbarkeit (Vorfilter)
for r in route_ids:
    for v in ev_types:
        if not ev_can_do_route(r, v):
            for k in range(MAX_VEH):
                prob += assign[r,v,k] == 0

# 2. Ladeinfrastruktur Limits
prob += lpSum([num_chg[c] for c in charger_types]) <= MAX_CHARGERS
prob += tot_cables == lpSum([num_chg[c] * chargers_data[c]['cables'] for c in charger_types])
prob += installed_pwr == lpSum([num_chg[c] * chargers_data[c]['power'] for c in charger_types])

# 3. EV Location & Charging Constraints
for v in ev_types:
    for k in range(MAX_VEH):
        for t in range(NUM_SLOTS):
            # Prüfen ob Fahrzeug auf Tour ist
            active_routes = [r for r in route_ids if routes_data[r]['t_start'] <= t < routes_data[r]['t_end']]
            if active_routes:
                is_driving = lpSum([assign[r,v,k] for r in active_routes])
                # Wenn fährt -> nicht am Hub
                prob += at_hub[v,k,t] == veh_exists[v,k] - is_driving
            else:
                # Sonst am Hub (wenn es existiert)
                prob += at_hub[v,k,t] == veh_exists[v,k]

            # Laden nur am Hub
            prob += plugged[v,k,t] <= at_hub[v,k,t]

            # NACHT-REGEL: 18:00 (Slot 72) bis 06:00 (Slot 24) -> SOFORT ANSTECKEN
            if t >= DAY_END or t < DAY_START:
                prob += plugged[v,k,t] == at_hub[v,k,t]

            # Ladeleistung Limit (Fahrzeugseitig)
            prob += ev_chg_pwr[v,k,t] <= vehicles_data[v]['charge_rate'] * plugged[v,k,t]

# Ladeleistung & Kabel gesamt Limit (Infrastrukturseitig)
for t in range(NUM_SLOTS):
    # Summe EV Strom <= Installierte Power
    prob += lpSum([ev_chg_pwr[v,k,t] for v in ev_types for k in range(MAX_VEH)]) <= installed_pwr
    # Summe Stecker <= Anzahl Kabel
    prob += lpSum([plugged[v,k,t] for v in ev_types for k in range(MAX_VEH)]) <= tot_cables

# 4. SoC Dynamik
for v in ev_types:
    cap = vehicles_data[v]['battery']
    cons_factor = vehicles_data[v]['consumption'] / 100.0 # kWh/km
    for k in range(MAX_VEH):
        for t in range(NUM_SLOTS):
            # Verbrauch berechnen
            active_routes = [r for r in route_ids if routes_data[r]['t_start'] <= t < routes_data[r]['t_end']]
            if active_routes:
                # Energie pro Slot = Gesamtenergie / Dauer
                energy_out = lpSum([assign[r,v,k] * routes_data[r]['miles'] * cons_factor / routes_data[r]['dur'] for r in active_routes])
            else:
                energy_out = 0

            # SoC Bilanz: Neu = Alt + (Laden - Fahren) / Kapazität
            prob += soc[v,k,t+1] == soc[v,k,t] + (ev_chg_pwr[v,k,t] * SLOT_DURATION - energy_out) / cap

        # Zyklischer SoC (Start = Ende)
        prob += soc[v,k,0] == soc[v,k,NUM_SLOTS]

# 5. Stationärer Speicher & Netz
for t in range(NUM_SLOTS):
    # Speicher Limit
    prob += batt_chg[t] <= batt_pwr
    prob += batt_dis[t] <= batt_pwr

    # Grid Bilanz
    # Netzbezug = EV Laden + Speicher Laden - Speicher Entladen
    total_ev_load = lpSum([ev_chg_pwr[v,k,t] for v in ev_types for k in range(MAX_VEH)])
    grid_draw = total_ev_load + batt_chg[t] - batt_dis[t]

    # Peak Tracking & Limit
    prob += peak_pwr >= grid_draw
    prob += grid_draw <= GRID_BASE + expand_grid * GRID_EXTRA

# Speicher SoC
for t in range(NUM_SLOTS):
    # Energiebilanz mit Wirkungsgrad (One-Way Efficiency applied to both directions for simplification of roundtrip)
    prob += batt_energy[t+1] == batt_energy[t] + (batt_chg[t] * BATT_EFF * SLOT_DURATION) - (batt_dis[t] / BATT_EFF * SLOT_DURATION)

    prob += batt_energy[t] <= batt_cap
    prob += batt_energy[t] >= batt_cap * MIN_SOC

prob += batt_energy[0] == batt_energy[NUM_SLOTS]

# ==========================================
# 4. LÖSUNG & AUSGABE
# ==========================================

print("Starte Solver (Zeitlimit 600s, Gap 5%)...")
solver = pulp.PULP_CBC_CMD(msg=1, timeLimit=300, gapRel=0.05)
prob.solve(solver)

print(f"\nStatus: {LpStatus[prob.status]}")

if prob.status == 1:
    total_tco = value(prob.objective)
    print("="*60)
    print(f"OPTIMALES ERGEBNIS - TCO: {total_tco:,.2f} Euro/Jahr")
    print("="*60)

    # 1. Flottenmix
    print("\n--- Fahrzeugflotte ---")
    for v in vehicle_types:
        count = value(num_veh[v])
        if count > 0:
            print(f"  {v}: {int(count)} Fahrzeuge")

    # 2. Infrastruktur
    print("\n--- Infrastruktur ---")
    for c in charger_types:
        count = value(num_chg[c])
        if count > 0:
            print(f"  Ladesäule {c}: {int(count)} Stück")

    if value(expand_grid) > 0.5:
        print(f"  Netzanschluss: ERWEITERT auf {GRID_BASE + GRID_EXTRA} kW")
    else:
        print(f"  Netzanschluss: Standard ({GRID_BASE} kW)")
    print(f"  Maximale Spitzenlast (Peak): {value(peak_pwr):.2f} kW")

    if value(batt_pwr) > 1 or value(batt_cap) > 1:
        print(f"  Batteriespeicher: {value(batt_pwr):.1f} kW / {value(batt_cap):.1f} kWh")
    else:
        print("  Batteriespeicher: Keine Installation")

    # 3. Kostenaufstellung
    print("\n--- Kostenaufstellung (jährlich) ---")
    cost_items = {
        "Flotte (Leasing/Wartung/Steuer)": value(fleet_cost),
        "Förderung (THG)": value(incentive) * -1,
        "Ladeinfrastruktur": value(chg_cost),
        "Netzanschluss (Grund + Erweit.)": value(grid_cost),
        "Batteriespeicher": value(batt_cost),
        "Diesel": value(fuel_cost),
        "Ladestrom": value(elec_cost),
        "Leistungspreis (Peak)": value(peak_cost),
        "Maut": value(toll_cost)
    }
    for item, val in cost_items.items():
        print(f"  {item:35s}: {val:>12,.2f} Euro")
    print("-" * 50)
    print(f"  {'SUMME':35s}: {total_tco:>12,.2f} Euro")

else:
    print("Keine optimale Lösung gefunden.")

import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, LpBinary, LpContinuous
from pulp import lpSum, LpStatus, value
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import math

warnings.filterwarnings('ignore')

print("Initialisiere Fleet Optimization Model (Strukturierte Indexmengen)...")

# ==========================================
# 0. DATEN-IMPORT (Raw Data)
# ==========================================

# Kosten & Technische Parameter
GAS_PRICE = 1.60        # Euro/Liter
ELEC_PRICE = 0.25       # Euro/kWh
BASE_ELEC_FEE = 1000    # Euro/Jahr
PEAK_FEE = 150          # Euro/kW/Jahr

GRID_BASE = 500         # kW
GRID_EXTRA = 500        # kW
GRID_EXPAND_COST = 10000 # Euro/Jahr

BATT_PWR_COST = 30      # Euro/kW/Jahr
BATT_CAP_COST = 350     # Euro/kWh/Jahr
BATT_MAINT = 0.02       # 2%
BATT_EFF_ROUND = 0.98
BATT_EFF = math.sqrt(BATT_EFF_ROUND)
MAX_DOD = 0.975
MIN_SOC = 1 - MAX_DOD

DAYS_PER_YEAR = 260
MAX_CHARGERS = 3

# Raw Data Dictionaries
routes_data = {
    't-4': {'miles': 250, 'toll_miles': 150, 'start': '06:45', 'end': '17:15'},
    't-5': {'miles': 250, 'toll_miles': 150, 'start': '06:30', 'end': '17:00'},
    't-6': {'miles': 250, 'toll_miles': 150, 'start': '06:00', 'end': '16:30'},
    's-1': {'miles': 120, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    's-2': {'miles': 120, 'toll_miles': 32, 'start': '06:00', 'end': '16:00'},
    's-3': {'miles': 120, 'toll_miles': 32, 'start': '09:00', 'end': '16:00'},
    's-4': {'miles': 120, 'toll_miles': 32, 'start': '06:30', 'end': '16:30'},
    'w1':  {'miles': 100, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    'w2':  {'miles': 100, 'toll_miles': 32, 'start': '08:00', 'end': '18:00'},
    'w3':  {'miles': 100, 'toll_miles': 32, 'start': '06:45', 'end': '16:45'},
    'w4':  {'miles': 100, 'toll_miles': 32, 'start': '06:00', 'end': '16:00'},
    'w5':  {'miles': 100, 'toll_miles': 32, 'start': '07:00', 'end': '17:00'},
    'w6':  {'miles': 100, 'toll_miles': 32, 'start': '05:30', 'end': '15:30'},
    'w7':  {'miles': 100, 'toll_miles': 32, 'start': '07:15', 'end': '17:15'},
    'r1':  {'miles': 285, 'toll_miles': 259, 'start': '18:00', 'end': '22:30'},
    'r2':  {'miles': 250, 'toll_miles': 220, 'start': '16:30', 'end': '21:45'},
    'r3':  {'miles': 235, 'toll_miles': 219, 'start': '17:45', 'end': '21:30'},
    'h3':  {'miles': 180, 'toll_miles': 160, 'start': '18:45', 'end': '22:45'},
    'h4':  {'miles': 180, 'toll_miles': 160, 'start': '18:30', 'end': '22:30'},
    'k1':  {'miles': 275, 'toll_miles': 235, 'start': '16:30', 'end': '22:30'},
}

vehicles_data = {
    'ActrosL':    {'is_ev': False, 'lease': 24000, 'maintenance': 6000, 'consumption': 26,  'tax': 556, 'toll_per_km': 0.34, 'battery': None, 'charge_rate': None, 'subsidy': 0},
    'eActros600': {'is_ev': True,  'lease': 60000, 'maintenance': 6000, 'consumption': 110, 'tax': 0,   'toll_per_km': 0,    'battery': 621,  'charge_rate': 400,  'subsidy': 1000},
    'eActros400': {'is_ev': True,  'lease': 50000, 'maintenance': 5000, 'consumption': 105, 'tax': 0,   'toll_per_km': 0,    'battery': 414,  'charge_rate': 400,  'subsidy': 1000}
}

chargers_data = {
    'Alpitronic-50':  {'invest': 3000,  'maint': 1000, 'power': 50,  'cables': 2},
    'Alpitronic-200': {'invest': 10000, 'maint': 1500, 'power': 200, 'cables': 2},
    'Alpitronic-400': {'invest': 16000, 'maint': 2000, 'power': 400, 'cables': 2}
}

# ==========================================
# 1. HELPER FUNCTIONS (Vorverarbeitung)
# ==========================================

def convert_time_to_slot(t_str):
    t_str = t_str.strip()
    try:
        if 'AM' in t_str or 'PM' in t_str: parsed = datetime.strptime(t_str, '%I:%M %p')
        else: parsed = datetime.strptime(t_str, '%H:%M')
        return (parsed.hour * 60 + parsed.minute) // 15
    except: return 0

# Zeiten in Routen konvertieren
for rid in routes_data:
    routes_data[rid]['t_start'] = convert_time_to_slot(routes_data[rid]['start'])
    routes_data[rid]['t_end'] = convert_time_to_slot(routes_data[rid]['end'])
    dur = routes_data[rid]['t_end'] - routes_data[rid]['t_start']
    routes_data[rid]['dur'] = max(dur, 1)

def check_overlap(r1, r2):
    s1, e1 = routes_data[r1]['t_start'], routes_data[r1]['t_end']
    s2, e2 = routes_data[r2]['t_start'], routes_data[r2]['t_end']
    return not (e1 <= s2 or e2 <= s1)

def ev_can_do_route(route_id, veh_type):
    if not vehicles_data[veh_type]['is_ev']: return True
    km = routes_data[route_id]['miles']
    needed = km * vehicles_data[veh_type]['consumption'] / 100.0
    return needed <= vehicles_data[veh_type]['battery'] * MAX_DOD


# ==========================================
# 2. DEFINITION DER INDEXMENGEN (SETS)
# ==========================================

# R: Menge der Routen
R = list(routes_data.keys())

# V: Menge der Fahrzeugtypen
V = list(vehicles_data.keys())

# Sub-Sets für V
V_EV = [v for v in V if vehicles_data[v]['is_ev']]
V_ICE = [v for v in V if not vehicles_data[v]['is_ev']]

# K: Menge der individuellen Fahrzeuge pro Typ (1...15)
# Dient als Zählindex, um mehrere Autos desselben Typs zu unterscheiden
MAX_VEH = 15
K = range(MAX_VEH)

# C: Menge der Ladesäulentypen
C = list(chargers_data.keys())

# T: Zeit-Slots
NUM_SLOTS = 96
SLOT_DURATION = 0.25
T = range(NUM_SLOTS)         # 0..95
T_SOC = range(NUM_SLOTS + 1) # 0..96 (für SoC Start/Ende)

# Zeit-Parameter in Slots
DAY_START = 24 # 06:00
DAY_END = 72   # 18:00


# ==========================================
# 3. MODELLIERUNG
# ==========================================

print("Erstelle Optimierungsmodell...")
prob = LpProblem("Fleet_Optimization_CaseStudy", LpMinimize)

# --- Entscheidungsvariablen ---

# Anzahl beschaffter Fahrzeuge pro Typ
num_veh = {v: LpVariable(f"num_{v}", lowBound=0, cat=LpInteger) for v in V}

# Binärvariable: Existiert Fahrzeug k vom Typ v?
veh_exists = {(v, k): LpVariable(f"exists_{v}_{k}", cat=LpBinary) for v in V for k in K}

# Binärvariable: Fährt Fahrzeug k vom Typ v die Route r?
assign = {(r, v, k): LpVariable(f"assign_{r}_{v}_{k}", cat=LpBinary)
          for r in R for v in V for k in K}

# Anzahl Ladesäulen pro Typ
num_chg = {c: LpVariable(f"nchg_{c}", lowBound=0, cat=LpInteger) for c in C}

# Infrastruktur
expand_grid = LpVariable("expand_grid", cat=LpBinary)
batt_pwr = LpVariable("batt_pwr", lowBound=0)
batt_cap = LpVariable("batt_cap", lowBound=0)
peak_pwr = LpVariable("peak_pwr", lowBound=0)

# Zeitreihen-Variablen
batt_energy = {t: LpVariable(f"batt_e_{t}", lowBound=0) for t in T_SOC}
batt_chg = {t: LpVariable(f"batt_ch_{t}", lowBound=0) for t in T}
batt_dis = {t: LpVariable(f"batt_dis_{t}", lowBound=0) for t in T}

# EV-Spezifische Variablen
soc = {}
ev_chg_pwr = {}
at_hub = {}
plugged = {}

for v in V_EV:
    for k in K:
        for t in T_SOC:
             soc[v,k,t] = LpVariable(f"soc_{v}_{k}_{t}", lowBound=MIN_SOC, upBound=1)
        for t in T:
            ev_chg_pwr[v,k,t] = LpVariable(f"evchg_{v}_{k}_{t}", lowBound=0)
            at_hub[v,k,t] = LpVariable(f"hub_{v}_{k}_{t}", cat=LpBinary)
            plugged[v,k,t] = LpVariable(f"plug_{v}_{k}_{t}", cat=LpBinary)

tot_cables = LpVariable("tot_cables", lowBound=0)
installed_pwr = LpVariable("inst_pwr", lowBound=0)

# --- Zielfunktion ---

# 1. Fixkosten Flotte
fleet_cost = lpSum([num_veh[v] * (vehicles_data[v]['lease'] + vehicles_data[v]['maintenance'] + vehicles_data[v]['tax']) for v in V])
incentive = lpSum([num_veh[v] * vehicles_data[v]['subsidy'] for v in V_EV])

# 2. Infrastruktur
chg_cost = lpSum([num_chg[c] * (chargers_data[c]['invest'] + chargers_data[c]['maint']) for c in C])
grid_cost = BASE_ELEC_FEE + expand_grid * GRID_EXPAND_COST
batt_cost = (batt_pwr * BATT_PWR_COST + batt_cap * BATT_CAP_COST) * (1 + BATT_MAINT)

# 3. Variable Kosten
# Diesel (Nur V_ICE)
fuel_cost = DAYS_PER_YEAR * lpSum([
    assign[r,v,k] * routes_data[r]['miles'] * vehicles_data[v]['consumption'] / 100.0 * GAS_PRICE
    for r in R for v in V_ICE for k in K
])

# Strom (Nur V_EV)
elec_cost = DAYS_PER_YEAR * ELEC_PRICE * lpSum([
    ev_chg_pwr[v,k,t] * SLOT_DURATION
    for v in V_EV for k in K for t in T
])

peak_cost = peak_pwr * PEAK_FEE

# Maut (Nur V_ICE)
toll_cost = DAYS_PER_YEAR * lpSum([
    assign[r,v,k] * routes_data[r]['toll_miles'] * vehicles_data[v]['toll_per_km']
    for r in R for v in V_ICE for k in K
])

prob += fleet_cost - incentive + chg_cost + grid_cost + batt_cost + fuel_cost + elec_cost + peak_cost + toll_cost

# --- Nebenbedingungen ---

# A. Routing & Flotte
# Jede Route muss von genau einem Fahrzeug gefahren werden
for r in R:
    prob += lpSum([assign[r,v,k] for v in V for k in K]) == 1

# Kopplung Anzahl Fahrzeuge <-> Existenz-Variable
for v in V:
    prob += lpSum([veh_exists[v,k] for k in K]) == num_veh[v]
    # Symmetrie-Brechung: Fahrzeug k nur existent, wenn k-1 auch existiert
    for k in range(len(K) - 1):
        prob += veh_exists[v,k] >= veh_exists[v,k+1]

# Zuordnung nur möglich, wenn Fahrzeug existiert
for r in R:
    for v in V:
        for k in K:
            prob += assign[r,v,k] <= veh_exists[v,k]

# Zeitliche Überschneidungen verhindern
for v in V:
    for k in K:
        for i, r1 in enumerate(R):
            for r2 in R[i+1:]:
                if check_overlap(r1, r2):
                    prob += assign[r1,v,k] + assign[r2,v,k] <= 1

# Reichweiten-Prüfung (Pre-Processing Constraint)
for r in R:
    for v in V_EV:
        if not ev_can_do_route(r, v):
            for k in K:
                prob += assign[r,v,k] == 0

# B. Ladeinfrastruktur (CapEx)
prob += lpSum([num_chg[c] for c in C]) <= MAX_CHARGERS
prob += tot_cables == lpSum([num_chg[c] * chargers_data[c]['cables'] for c in C])
prob += installed_pwr == lpSum([num_chg[c] * chargers_data[c]['power'] for c in C])

# C. EV-Logik (OpEx)
for v in V_EV:
    for k in K:
        for t in T:
            # Ist Fahrzeug auf Tour?
            active_routes = [r for r in R if routes_data[r]['t_start'] <= t < routes_data[r]['t_end']]
            if active_routes:
                is_driving = lpSum([assign[r,v,k] for r in active_routes])
                prob += at_hub[v,k,t] == veh_exists[v,k] - is_driving
            else:
                prob += at_hub[v,k,t] == veh_exists[v,k]

            # Anstecken nur möglich wenn am Hub
            prob += plugged[v,k,t] <= at_hub[v,k,t]

            # NACHT-REGEL: 18:00 - 06:00 Uhr Pflicht zum Anstecken
            if t >= DAY_END or t < DAY_START:
                prob += plugged[v,k,t] == at_hub[v,k,t]

            # Ladeleistung pro Fahrzeug
            prob += ev_chg_pwr[v,k,t] <= vehicles_data[v]['charge_rate'] * plugged[v,k,t]

# D. Infrastruktur Limits (Aggregiert)
for t in T:
    # Summe aller Ladeleistungen <= Installierte Säulenleistung
    prob += lpSum([ev_chg_pwr[v,k,t] for v in V_EV for k in K]) <= installed_pwr
    # Anzahl steckender Fahrzeuge <= Anzahl Kabel
    prob += lpSum([plugged[v,k,t] for v in V_EV for k in K]) <= tot_cables

# E. SoC Bilanz (State of Charge)
for v in V_EV:
    cap = vehicles_data[v]['battery']
    cons_factor = vehicles_data[v]['consumption'] / 100.0
    for k in K:
        for t in T:
            active_routes = [r for r in R if routes_data[r]['t_start'] <= t < routes_data[r]['t_end']]
            if active_routes:
                # Energieverbrauch während der Fahrt
                energy_out = lpSum([assign[r,v,k] * routes_data[r]['miles'] * cons_factor / routes_data[r]['dur'] for r in active_routes])
            else:
                energy_out = 0

            # SoC t+1 = SoC t + (Laden - Verbrauch) / Kapazität
            prob += soc[v,k,t+1] == soc[v,k,t] + (ev_chg_pwr[v,k,t] * SLOT_DURATION - energy_out) / cap

        # Zyklischer Tagesablauf
        prob += soc[v,k,0] == soc[v,k,NUM_SLOTS]

# F. Speicher & Netzanschluss
for t in T:
    prob += batt_chg[t] <= batt_pwr
    prob += batt_dis[t] <= batt_pwr

    total_ev_load = lpSum([ev_chg_pwr[v,k,t] for v in V_EV for k in K])
    grid_draw = total_ev_load + batt_chg[t] - batt_dis[t]

    prob += peak_pwr >= grid_draw
    prob += grid_draw <= GRID_BASE + expand_grid * GRID_EXTRA

# Speicher SoC
for t in T:
    prob += batt_energy[t+1] == batt_energy[t] + (batt_chg[t] * BATT_EFF * SLOT_DURATION) - (batt_dis[t] / BATT_EFF * SLOT_DURATION)
    prob += batt_energy[t] <= batt_cap
    prob += batt_energy[t] >= batt_cap * MIN_SOC

prob += batt_energy[0] == batt_energy[NUM_SLOTS]

# ==========================================
# 4. SOLVER & AUSGABE
# ==========================================

print("Starte Solver (Zeitlimit 300s, Gap 5%)...")
solver = pulp.PULP_CBC_CMD(msg=1, timeLimit=36000, gapRel=0.05)
prob.solve(solver)

print(f"\nStatus: {LpStatus[prob.status]}")

if prob.status == 1:
    total_tco = value(prob.objective)
    print("="*60)
    print(f"OPTIMALES ERGEBNIS - TCO: {total_tco:,.2f} Euro/Jahr")
    print("="*60)

    print("\n--- Fahrzeugflotte ---")
    for v in V:
        count = value(num_veh[v])
        if count > 0: print(f"  {v}: {int(count)} Fahrzeuge")

    print("\n--- Infrastruktur ---")
    for c in C:
        count = value(num_chg[c])
        if count > 0: print(f"  Ladesäule {c}: {int(count)} Stück")

    if value(expand_grid) > 0.5: print(f"  Netzanschluss: ERWEITERT auf {GRID_BASE + GRID_EXTRA} kW")
    else: print(f"  Netzanschluss: Standard ({GRID_BASE} kW)")
    print(f"  Maximale Spitzenlast (Peak): {value(peak_pwr):.2f} kW")

    if value(batt_pwr) > 1 or value(batt_cap) > 1:
        print(f"  Batteriespeicher: {value(batt_pwr):.1f} kW / {value(batt_cap):.1f} kWh")
    else: print("  Batteriespeicher: Keine Installation")

    print("\n--- Kostenaufstellung (jährlich) ---")
    cost_items = {
        "Flotte (Leasing/Wartung/Steuer)": value(fleet_cost),
        "Förderung (THG)": value(incentive) * -1,
        "Ladeinfrastruktur": value(chg_cost),
        "Netzanschluss": value(grid_cost),
        "Batteriespeicher": value(batt_cost),
        "Diesel": value(fuel_cost),
        "Ladestrom": value(elec_cost),
        "Leistungspreis": value(peak_cost),
        "Maut": value(toll_cost)
    }
    for item, val in cost_items.items():
        print(f"  {item:35s}: {val:>12,.2f} Euro")
    print("-" * 50)
    print(f"  {'SUMME':35s}: {total_tco:>12,.2f} Euro")
else:
    print("Keine optimale Lösung gefunden.")


Initialisiere Fleet Optimization Model...
Erstelle Optimierungsmodell...
Starte Solver (Zeitlimit 600s, Gap 5%)...

Status: Optimal
OPTIMALES ERGEBNIS - TCO: 971,159.60 Euro/Jahr

--- Fahrzeugflotte ---
  ActrosL: 14 Fahrzeuge

--- Infrastruktur ---
  Netzanschluss: Standard (500 kW)
  Maximale Spitzenlast (Peak): 0.00 kW
  Batteriespeicher: Keine Installation

--- Kostenaufstellung (jährlich) ---
  Flotte (Leasing/Wartung/Steuer)    :   427,784.00 Euro
  Förderung (THG)                    :        -0.00 Euro
  Ladeinfrastruktur                  :         0.00 Euro
  Netzanschluss (Grund + Erweit.)    :     1,000.00 Euro
  Batteriespeicher                   :         0.00 Euro
  Diesel                             :   360,713.60 Euro
  Ladestrom                          :         0.00 Euro
  Leistungspreis (Peak)              :         0.00 Euro
  Maut                               :   181,662.00 Euro
--------------------------------------------------
  SUMME                            